In [ ]:
# !pip install praw
# !pip install vaderSentiment

In [ ]:
import pandas as pd

import praw
import requests

import datetime
from datetime import datetime
from dateutil import parser
from collections import Counter
import time
import re


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score,confusion_matrix, f1_score, precision_score, recall_score


from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from textblob import TextBlob

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import AutoModelForSequenceClassification, DistilBertForSequenceClassification, BertForSequenceClassification
from transformers import AutoTokenizer, DistilBertTokenizer, BertTokenizer
from transformers import Trainer, TrainingArguments

import torch
from torch.utils.data import DataLoader


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

REDDIT API SCRAPING

In [ ]:
#Initialize reddit API client
reddit_api = praw.Reddit( client_id = "O-e6Uhhw2QSiPdkDeOixig",
                    client_secret = "PFItTkmpzDi2XI8rmzFBy3pJq0RpwQ",
                    user_agent = "Crypto_Analysis"
                    )


In [ ]:
#Define subreddit() to scrape posts on tiopic 'Bitcoin'
bitcoin_subreddit = reddit_api.subreddit("Bitcoin")

# Scraping the top posts of all time from the subreddit
posts = bitcoin_subreddit.top(time_filter = "all", limit = None)

#Define dictionary to store scraped data
posts_dict = {"Title": [], "Post Text": [],
              "ID": [], "Score": [], "Upvote Ratio": [],
              "Total Comments": [],"Created On":[], "Post URL": [],
              "Original Content": [],
              }

#From-date for scraping
start_date = '01-01-18 00:00:00'
start_date = datetime.strptime(start_date, '%d-%m-%y %H:%M:%S').timestamp()

#Iterate through each post and extracting relevant data
for post in posts:
    # Date of each posts' creation
    date = post.created_utc
    #Checking if the post is after start_date
    if date > start_date:
        #Appending relevant data to dictionary
        # Title of each post
        posts_dict["Title"].append(post.title)

        # Text inside a post
        posts_dict["Post Text"].append(post.selftext)

        # Unique ID of each post
        posts_dict["ID"].append(post.id)

        # The score of a post
        posts_dict["Score"].append(post.score)

        # Upvote Ratio of a post
        posts_dict["Upvote Ratio"].append(post.upvote_ratio)

        # Total number of comments inside the post
        posts_dict["Total Comments"].append(post.num_comments)

        # Date the post was Created
        posts_dict["Created On"].append(post.created_utc)

        # URL of each post
        posts_dict["Post URL"].append(post.url)

        # Flair of each post
        posts_dict["Original Content"].append(post.is_original_content)


# Saving the dictionary as dataframe
all_posts = pd.DataFrame(posts_dict)
#Convert timestamp to datetime format
all_posts['Created On'] = pd.to_datetime(all_posts['Created On'],  unit='s')


In [ ]:
all_posts

,Title,Post Text,ID,Score,Upvote Ratio,Total Comments,Created On,Post URL,Original Content
0,The last 3 months in 47 seconds.,,7v438b,48542,0.90,761,2018-02-04 02:42:03,https://v.redd.it/typ8fdslz3e01,False
1,Tesla buys $1.5b in Bitcoin and is looking to ...,,lfb3mm,45539,0.87,2857,2021-02-08 12:48:45,https://v.redd.it/gazga81l29g61,False
2,Everyone who's trading BTC right now,,7olruz,42077,0.88,760,2018-01-06 20:38:56,http://cdn.mutually.com/wp-content/uploads/201...,False
3,Robinhood just blocked several stocks from bei...,Anyone following the WSB drama this morning wi...,l70vrq,39990,0.96,1544,2021-01-28 16:11:32,https://www.reddit.com/r/Bitcoin/comments/l70v...,False
4,"Quick, upvote this to confuse everyone into th...",,chbx9d,36863,0.84,566,2019-07-24 18:24:16,https://i.redd.it/pcioshemnac31.gif,False
...,...,...,...,...,...,...,...,...,...
803,You can mine bitcoin on a Gameboy,,x3bjkm,3057,0.97,370,2022-09-01 16:09:26,https://i.redd.it/fkszxlk6u9l91.jpg,False
804,Central banks doing what they do best.,,hpxa9g,3054,0.92,331,2020-07-12 16:33:49,https://i.redd.it/vns3475mega51.png,False
805,"Cathie Wood Ark CEO: Bitcoin is the ""Best Hedg...",,ls3udq,3049,0.95,405,2021-02-25 10:44:46,https://youtu.be/4GXd_XD6ivM,False
806,So i found an old bitcoin wallet from 2009!,I go thru old hardrives people have thrown awa...,1dclonn,3052,0.95,815,2024-06-10 13:36:33,https://i.redd.it/yionalg4zq5d1.png,False


# Analysing the performance of different sentiment analyzer tools using a manually labeled dataset

In [ ]:
#Dataframe to store the accuracies for each model
sentiment_analyser_comparison = pd.DataFrame()

In [1]:
# Define the file path
sentiment_analyser_comparison_path = 'data/sentiment_analyser_model_comparison_reddit.csv'


VADER SENTIMENT ANALYSIS

In [ ]:
def vader_sentiment_analysis(df,column=1):

  """
    Takes a DataFrame of reddit details, analyze sentiment for each reddit and is added to a list which is returned

    Parameters:
    df(DataFrame): DataFrame with reddit dataset

    Retuns:
    vader_sentiments(List) : List of sentiments for each row
  """
  vader_sentiments = []
  for index,row in df.iterrows():
      # Create a SentimentIntensityAnalyzer object.
      vader = SentimentIntensityAnalyzer()

      # polarity_scores method of SentimentIntensityAnalyzer
      # object gives a sentiment dictionary.
      # which contains pos, neg, neu, and compound scores.
      vader_sentiment_dict = vader.polarity_scores(row[column])
      vader_sentiment = ""

      # decide sentiment as positive, negative and neutral by comparing to a threshold
      if vader_sentiment_dict['compound'] >= 0.05 :
          vader_sentiment = "Bullish"

      elif vader_sentiment_dict['compound'] <= - 0.05 :
          vader_sentiment = "Bearish"
      else :
          vader_sentiment = "Neutral"

      vader_sentiments.append(vader_sentiment)

  return vader_sentiments

TEXTBLOB


In [ ]:
def textblob_sentiment(df,column=1):
  """
    Analyzes the sentiment of text data in a DataFrame column using TextBlob and classifies
    the sentiment as 'Bearish', 'Bullish', or 'Neutral' based on the polarity score.

    Parameters:
    ----------
    df : pandas.DataFrame
        The DataFrame containing the dataset with text data to analyze.

    column : int or str, optional (default=1)
        The index or name of the column in the DataFrame that contains the text data for sentiment analysis.

    Returns:
    -------
    textblob_sentiments : list
        A list of sentiment classifications ('Bearish', 'Bullish', or 'Neutral') for each row in the DataFrame.


  """
  # Initialize an empty list to store the sentiment results.
  textblob_sentiments = []
  # Iterate over each row in the DataFrame.
  for index,row in df.iterrows():
    # Create a TextBlob object for the text in the specified column of the current row.
    textblob_analysis = TextBlob(row[column])
    # Analyze the polarity and classify the sentiment accordingly.
    if textblob_analysis.sentiment.polarity > 0.5:
      sentiment = "Bearish"
    elif textblob_analysis.sentiment.polarity < -0.5:
      sentiment = "Bullish"
    else:
      sentiment = "Neutral"

    # Append the classified sentiment to the list.
    textblob_sentiments.append(sentiment)

  # Return the list of sentiments.
  return textblob_sentiments


BERT SENTIMENT ANALYSIS

In [ ]:
def preprocess(texts,tokenizer):
    """
    Pre processing a list of texts for model input

    Parameters:
    texts(str):List of strings to tokenize

    Returns:
    inputs: Tokenized inputs for the model
    """
    #Tokenize the list of strings
    inputs = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    return inputs



In [ ]:

def bert_sentiment_analysis(df, column, bert_model=None, tokenizer=None):
    """
    Takes a dataframe and returns sentiment for each text using BERT model

    Parameters:
    df(pd.DataFrame()): Dataframe containing texts for analysing sentiment
    column(int): The column index in the DataFrame containing texts for sentiment analysis

    Returns:
    bert_sentiments(list): List of sentiment labels corresponding to texts
    """
    batch_size = 100
    bert_sentiments = []
    bert_label_map = {0: "Bearish", 1: "Neutral", 2: "Bullish"}
    for i in range(0,len(df),batch_size):
        #Taking strings in dataframe as a batch of size 'batch_size' and storing as list
        texts = df[i:i + batch_size][column].to_list()
        #Converting the list of strings as tokens
        inputs = preprocess(texts,tokenizer)
        #Analysing sentiment for the inputs using CryptoBERT model
        with torch.no_grad():
            outputs = bert_model(**inputs)

        # Get the logits from model output
        logits = outputs.logits
        #Getting probabilities from logits using softmax function
        probabilities = torch.nn.functional.softmax(logits, dim=-1)
        #Getting the sentiment index with higest probability
        labels = torch.argmax(probabilities, dim=1)
        #Mapping the label for each sentiment index
        for label in labels:
            bert_sentiments.append(bert_label_map[label.item()])
    return bert_sentiments

DATA PRE-PROCESSING

In [ ]:
def remove_tags(string):
    result = re.sub('','',string)          #remove HTML tags
    result = re.sub('https?://\S+|www\.\S+','',result)   #remove URLs
    result = re.sub(r'[^a-zA-Z\s]', ' ',result)    #remove non-alphanumeric characters
    result = result.lower()
    return result


In [ ]:
def stop_words_removal(text):
    """
    Takes a sentence and removes stop-words
    Parameters:
    text(str): Sentence from which stopwords are to be removed

    Returns:
    result_text(str): Sentence after removing stop words

    """
    # Initialising 'stopwords'
    stop_words = set(stopwords.words('english'))
    #Iterating through each word, filters stop words and append to result)text list
    result_text = [word for word in text.split() if word.lower() not in stop_words]
    return " ".join(result_text)

In [ ]:

def lemmatize_text(text):
    """
    Takes a text string and returns a lemmatized text

    Parameters:
    text(str): Text which is to be lemmatized

    Returns:
    lemmatized_text(str): Lemmatized string

    """
    #Initialize whitespace tokenizer
    w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
    #Initialise lemmatizer
    lemmatizer = nltk.stem.WordNetLemmatizer()
    #Tokenize the text using WhitespaceTokenizer()
    tokenized_text = w_tokenizer.tokenize(text)
    lemmatized_text = ""
    #Iterate through each word,lemmatizes and join back as sentence
    for word in tokenized_text:
        lemmatized_text = lemmatized_text + lemmatizer.lemmatize(word) + " "
    return lemmatized_text


MANUALLY LABELED DATASET

In [ ]:
reddit_labeled_path = 'data/reddit_manually_labeled_sentiments.csv'
reddit_predicted_sentiment_path = 'data/reddit_predicted_sentiment.csv'

In [ ]:
#Manualy labeled reddit posts
labeled_reddit = pd.read_csv(reddit_labeled_path)

In [ ]:
label = {
    'Bearish': 0,
    'Neutral': 1,
    'Bullish' : 2
}
labeled_reddit['actual_sentiment_num'] = labeled_reddit['actual_sentiment'].map(label) #The sentiments are mapped to Bearish, Neutral and Bullish.

In [ ]:
labeled_reddit['processed_reddit']=labeled_reddit['Title'].apply(lambda cw : remove_tags(cw))
labeled_reddit['processed_reddit'] = labeled_reddit['processed_reddit'].apply(stop_words_removal)
labeled_reddit['processed_reddit'] = labeled_reddit['processed_reddit'].apply(lemmatize_text)

In [ ]:
labeled_reddit

,Unnamed: 0,Title,actual_sentiment,actual_sentiment_num,processed_reddit
0,553,Friendly reminder: bitcoin wasn’t created to m...,Bullish,2,friendly reminder bitcoin created make rich cr...
1,199,1 Bitcoin just helped me put 50k down on my fi...,Bullish,2,bitcoin helped put k first home
2,119,The Colonel of KFC weighs in on alt-coins,Neutral,1,colonel kfc weighs alt coin
3,76,Sell your Bitcoin if you're going to be having...,Bullish,2,sell bitcoin going hemorrhoid every time bitco...
4,188,Xbox has put out a survey to get a feel for us...,Bullish,2,xbox put survey get feel using bitcoin purchas...
...,...,...,...,...,...
95,467,Finally made it to 1 complete Bitcoin.,Bullish,2,finally made complete bitcoin
96,63,The last few weeks in a nutshell,Neutral,1,last week nutshell
97,729,Always!,Neutral,1,always
98,131,Truer words were never written.,Neutral,1,truer word never written


SENTIMENT ANALYSIS ON RAW REDDIT POSTS

VADER model

In [ ]:
labeled_reddit['vader_sentiment_raw_reddit'] = vader_sentiment_analysis(labeled_reddit,'Title')

In [ ]:
#Storing accuracy of VADER model on raw reddit posts
vader_accuracy_raw_reddit = pd.DataFrame({
    'Model': ['VADER on raw Reddit posts'],
    'Accuracy': [accuracy_score(labeled_reddit['actual_sentiment'], labeled_reddit['vader_sentiment_raw_reddit'])]
})

# Use pd.concat() to add the new row
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, vader_accuracy_raw_reddit], ignore_index=True)


TextBlob

In [ ]:
labeled_reddit['textblob_sentiment_raw_reddit'] = textblob_sentiment(labeled_reddit,"Title")

In [ ]:
#Storing accuracy of TextBlob model on raw Reddit posts
textblob_accuracy_raw_reddit = pd.DataFrame({
    'Model': ['TextBlob on raw Reddit posts'],
    'Accuracy': [accuracy_score(labeled_reddit['actual_sentiment'], labeled_reddit['textblob_sentiment_raw_reddit'])]
})

# Use pd.concat() to add the new row
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, textblob_accuracy_raw_reddit], ignore_index=True)


TF-IDF approach

In [ ]:
# Initialize the TF-IDF vectorizer.
tfidfconverter = TfidfVectorizer()

# Fit the vectorizer on the 'raw_reddit' column of the DataFrame
# and transform the text into TF-IDF features.
X = tfidfconverter.fit_transform(labeled_reddit['Title'])
# Define the target variable (actual sentiment) for classification.
y = labeled_reddit['actual_sentiment_num']
# Split the data into training and testing sets (80% train, 20% test).
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
# Initialize the logistic regression model.
logistic_model_tfidf_raw_reddit = LogisticRegression()
# Train the logistic regression model using the training data.
logistic_model_tfidf_raw_reddit.fit(X_train,y_train)
# Make predictions on the test set.
y_pred = logistic_model_tfidf_raw_reddit.predict(X_test)
# Calculate the accuracy of the model's predictions using the test data.
tfidf_accuracy_raw_reddit = accuracy_score(y_test,y_pred)


In [ ]:
#Storing accuracy of TF-IDF approach on raw reddit posts
tfidf_accuracy_raw_reddit = pd.DataFrame({
    'Model': ['TF-IDF on raw Reddit posts'],
    'Accuracy': tfidf_accuracy_raw_reddit
})

# Use pd.concat() to add the new row
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, tfidf_accuracy_raw_reddit], ignore_index=True)


BERT models

In [ ]:
#Loading pre-trained CryptoBERT tokenizer for the model
cryptobert_tokenizer = AutoTokenizer.from_pretrained("ElKulako/cryptobert")
#Loading pre-trained CryptoBERT model for classification
cryptobert_model = AutoModelForSequenceClassification.from_pretrained("ElKulako/cryptobert")


#Loading pre-trained FinBERT tokenizer for the model
finbert_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
#Loading pre-trained FinBERT model for classification
finbert_model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')


#Loading pre-trained DistilBERT tokenizer for the model
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
#Loading pre-trained DistilBERT model for classification
distilbert_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
labeled_reddit['cryptobert_sentiment_raw_reddit']= bert_sentiment_analysis(labeled_reddit,"Title", cryptobert_model, cryptobert_tokenizer)
labeled_reddit['finbert_sentiment_raw_reddit'] = bert_sentiment_analysis(labeled_reddit,"Title", finbert_model, finbert_tokenizer)
labeled_reddit['distilbert_sentiment_raw_reddit'] = bert_sentiment_analysis(labeled_reddit,"Title", distilbert_model, distilbert_tokenizer)


In [ ]:
#Storing accuracy of CryptoBERT model on raw reddit posts
cryptobert_accuracy_raw_reddit = pd.DataFrame({
    'Model':[ 'CryptoBERT on raw Reddit posts'],
    "Accuracy": accuracy_score(labeled_reddit['actual_sentiment'], labeled_reddit['cryptobert_sentiment_raw_reddit'])
})

#Storing accuracy of FinBERT model on raw reddit posts
finbert_accuracy_raw_reddit = pd.DataFrame({
    'Model':[ 'FinBERT on raw Reddit posts'],
    "Accuracy": accuracy_score(labeled_reddit['actual_sentiment'], labeled_reddit['finbert_sentiment_raw_reddit'])
})

#Storing accuracy of DistilBERT model on raw reddit posts
distilbert_accuracy_raw_reddit = pd.DataFrame({
    'Model':[ 'DistilBERT on raw Reddit posts'],
    "Accuracy": accuracy_score(labeled_reddit['actual_sentiment'], labeled_reddit['distilbert_sentiment_raw_reddit'])
})


# Use pd.concat() to add the new row
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, cryptobert_accuracy_raw_reddit], ignore_index=True)
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, finbert_accuracy_raw_reddit], ignore_index=True)
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, distilbert_accuracy_raw_reddit], ignore_index=True)


## SENTIMENT ANALYSIS ON PROCESSED reddit

VADER model

In [ ]:
labeled_reddit['vader_sentiment_processed_reddit']= vader_sentiment_analysis(labeled_reddit,"processed_reddit")

In [ ]:
#Storing accuracy of VADER model on processed reddit posts
vader_accuracy_processed_reddit = pd.DataFrame({
    'Model': ['VADER on processed Reddit posts'],
    'Accuracy': [accuracy_score(labeled_reddit['actual_sentiment'], labeled_reddit['vader_sentiment_processed_reddit'])]
})

# Use pd.concat() to add the new row
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, vader_accuracy_processed_reddit], ignore_index=True)


TextBlob

In [ ]:
labeled_reddit['textblob_sentiment_processed_reddit']= textblob_sentiment(labeled_reddit,"processed_reddit")

In [ ]:
#Storing accuracy of TextBlob model on processed reddit posts
textblob_accuracy_processed_reddit = pd.DataFrame({
    'Model': ['TextBlob on processed Reddit posts'],
    'Accuracy': [accuracy_score(labeled_reddit['actual_sentiment'], labeled_reddit['textblob_sentiment_processed_reddit'])]
})

# Use pd.concat() to add the new row
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, textblob_accuracy_processed_reddit], ignore_index=True)


TF-IDF Approach

In [ ]:
# Initialize the TF-IDF vectorizer.
tfidfconverter = TfidfVectorizer()

# Fit the vectorizer on the 'processed_reddit' column of the DataFrame
# and transform the text into TF-IDF features.
X = tfidfconverter.fit_transform(labeled_reddit['processed_reddit'])
# Define the target variable (actual sentiment) for classification.
y = labeled_reddit['actual_sentiment_num']
# Split the data into training and testing sets (80% train, 20% test).
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)
# Initialize the logistic regression model.
logistic_model_tfidf_processed_reddit = LogisticRegression()
# Train the logistic regression model using the training data.
logistic_model_tfidf_processed_reddit.fit(X_train,y_train)
# Make predictions on the test set.
y_pred = logistic_model_tfidf_processed_reddit.predict(X_test)
# Calculate the accuracy of the model's predictions using the test data.
tfidf_accuracy_processed_reddit = accuracy_score(y_test,y_pred)


In [ ]:
#Storing accuracy of TF-IDF approach on processed reddit
tfidf_accuracy_processed_reddit = pd.DataFrame({
    'Model': ['TF-IDF on processed reddit'],
    'Accuracy': tfidf_accuracy_processed_reddit
})

# Use pd.concat() to add the new row
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, tfidf_accuracy_processed_reddit], ignore_index=True)


BERT models

In [ ]:
#Loading pre-trained CryptoBERT tokenizer for the model
cryptobert_tokenizer = AutoTokenizer.from_pretrained("ElKulako/cryptobert")
#Loading pre-trained CryptoBERT model for classification
cryptobert_model = AutoModelForSequenceClassification.from_pretrained("ElKulako/cryptobert")


#Loading pre-trained FinBERT tokenizer for the model
finbert_tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
#Loading pre-trained FinBERT model for classification
finbert_model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone')


#Loading pre-trained DistilBERT tokenizer for the model
distilbert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
#Loading pre-trained DistilBERT model for classification
distilbert_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
labeled_reddit['cryptobert_sentiment_processed_reddit']= bert_sentiment_analysis(labeled_reddit,"processed_reddit", cryptobert_model, cryptobert_tokenizer)
labeled_reddit['finbert_sentiment_processed_reddit'] = bert_sentiment_analysis(labeled_reddit,"processed_reddit", finbert_model, finbert_tokenizer)
labeled_reddit['distilbert_sentiment_processed_reddit'] = bert_sentiment_analysis(labeled_reddit,"processed_reddit", distilbert_model, distilbert_tokenizer)


In [ ]:
#Storing accuracy of CryptoBERT model on processed reddit posts
cryptobert_accuracy_processed_reddit = pd.DataFrame({
    'Model':[ 'CryptoBERT on processed Reddit posts'],
    "Accuracy": accuracy_score(labeled_reddit['actual_sentiment'], labeled_reddit['cryptobert_sentiment_processed_reddit'])
})

#Storing accuracy of FinBERT model on processed reddit posts
finbert_accuracy_processed_reddit = pd.DataFrame({
    'Model':[ 'FinBERT on processed Reddit posts'],
    "Accuracy": accuracy_score(labeled_reddit['actual_sentiment'], labeled_reddit['finbert_sentiment_processed_reddit'])
})

#Storing accuracy of DistilBERT model on processed reddit posts
distilbert_accuracy_processed_reddit = pd.DataFrame({
    'Model':[ 'DistilBERT on processed Reddit posts'],
    "Accuracy": accuracy_score(labeled_reddit['actual_sentiment'], labeled_reddit['distilbert_sentiment_processed_reddit'])
})


# Use pd.concat() to add the new row
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, cryptobert_accuracy_processed_reddit], ignore_index=True)
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, finbert_accuracy_processed_reddit], ignore_index=True)
sentiment_analyser_comparison = pd.concat([sentiment_analyser_comparison, distilbert_accuracy_processed_reddit], ignore_index=True)


In [ ]:
sentiment_analyser_comparison

,Model,Accuracy
0,VADER on raw Reddit posts,0.62
1,TextBlob on raw Reddit posts,0.48
2,TF-IDF on raw Reddit posts,0.50
3,CryptoBERT on raw Reddit posts,0.40
4,FinBERT on raw Reddit posts,0.08
5,DistilBERT on raw Reddit posts,0.48
6,VADER on processed Reddit posts,0.60
7,TextBlob on processed Reddit posts,0.49
8,TF-IDF on processed reddit,0.60
9,CryptoBERT on processed Reddit posts,0.42


In [ ]:
# Save the DataFrame as a CSV
sentiment_analyser_comparison.to_csv(sentiment_analyser_comparison_path, index=False)

ANALYSING SENTIMENT FOR REDDIT POSTS WITH VADER

In [ ]:
# Analyse VADER model after data pre-processing
for index,row in all_posts.iterrows():
    reddit_vader_sentiment = vader_sentiment_scores(row['Title'])
    all_posts.at[index,'vader_sentiment'] = reddit_vader_sentiment

In [ ]:
all_posts['vader_sentiment']= vader_sentiment_analysis(labeled_reddit,"Title")

In [ ]:
all_posts.to_csv(reddit_predicted_sentiment_path)